In [5]:
%reload_ext autoreload
%autoreload 2

In [6]:
from tqdm.notebook import tqdm
from result_saver import SaverProvider
from Scratch import metadata_loader
provider = SaverProvider()

In [7]:
df = metadata_loader(_extract=True)
#df = df[df.job_status != 'JobStatus.QUEUED']


In [8]:
from time import sleep

start_idx = 0 # Initialize starting index

while True:  # Outer loop to restart when a specific error occurs
    try:
        for idx, job_id in enumerate(tqdm(df.job_id[start_idx:])):

            provider.retrieve_job(job_id, _ignore_running=False)

            start_idx = start_idx + 1  
        break  
    except KeyboardInterrupt:
        print('Interrupted')
        break  
    except Exception as e:
        if 'RequestsApiError' in str(type(e)):
            print(f'\nRequestsApiError occurred at job index {start_idx}, retrying...')
            sleep(5) 
        else:
            print(f'An unexpected error occurred: {e}')
            break  


  0%|          | 0/1017 [00:00<?, ?it/s]

17:44:48 Warning: Job ID cn2b4wkrmwhg008jxgyg not found in /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
17:45:05 Warning: Job ID cn2b4marmwhg008jxgy0 not found in /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 77
17:45:29 Warning: Job ID cn2b4bhvayrg008egp4g not found in /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider.... IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft

In [ ]:
df = df[df['job_id']=="cmxnz9dwjay0008cnwc0"]
if df.job_status.values == 'JobStatus.DONE':
    print("Job is done")    